In [1]:
from collections import defaultdict

from pathlib import Path
import os
import sys
import json
import pickle
#os.environ['CUDA_VISIBLE_DEVICES']='0'  # don't occupy GPU, inference can work on CPU
import torch
import logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from typing import List, Dict, Tuple

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
from sklearn.pipeline import Pipeline

# from python_ev.dataset import load_parts, load_df
# from vectorizers.bert_boris.bert_base_spwsi import BertBaseSPWSI
# from subtask1_solutions.base_solution import Subtask1BaseVectorizerFactory
# import flair

02/21/2019 16:27:43 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English


In [2]:
def gen_list_of_inputs(sent, tok = 'MASK', lemma = ''):
    tokens = sent.split(' ')
    res = []
    for i in range(0, len(tokens) + 1):
        tokens = sent.split(' ')
        tokens.insert(i, tok)
        tokens.insert(i+1, tok) # убрать
        res.append({str(i):(tokens, i, lemma)})
    return res

In [3]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [4]:
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case = False)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
      

In [5]:
text = "Я думаю, она вскоре будет на ногах и через несколько дней на пути к выздоровлению."

In [6]:
tok = tokenizer.tokenize(text)

In [7]:
tok

['Я',
 'ду',
 '##ма',
 '##ю',
 ',',
 'она',
 'вскоре',
 'будет',
 'на',
 'но',
 '##гах',
 'и',
 'через',
 'несколько',
 'дней',
 'на',
 'пути',
 'к',
 'вы',
 '##здо',
 '##ров',
 '##лению',
 '.']

In [8]:
tok.insert(17, '[MASK]')

In [9]:
tok.insert(0, '[MASK]')

In [10]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tok)

In [11]:
segments_ids = [0 for x in range(len(indexed_tokens))]

In [12]:
tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
segments_tensors = torch.tensor([segments_ids]).to('cuda')
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
      

In [13]:
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]], device='cuda:0')

In [14]:
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

In [15]:
predicted_index = torch.argmax(predictions[0, 17]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

In [16]:
predicted_token

'пути'

In [17]:
def bottom_layer(tok_sent, verb_list, tokenizer, index, model, threshold):
    # получаем предложение, список глаголов, токенизатор, индекс конца слова; возвращаем "похожие" токенизированные слова
    # также подаём на вход модель; threshold - количество получаемых элементов
    #model.to('cuda')
    #tok_sent = tokenizer.tokenize(sent) # пока что
    res = {}
    if (len(verb_list) == 1):
        verb = verb_list[0] # список из k элементов (k - количество токенов)
        num_tokens = len(verb)
        for i in range(index+2, len(tok_sent)+1):
            if (i == len(tok_sent) or tok_sent[i].isalpha() or tok_sent[i].isdigit()):
                new_tok = tok_sent.copy()
                print(tok_sent[i-1])
                print('=====')
                # тогда перед этим всем вставляем num_tokens токенов
                for _ in range(num_tokens):
                    new_tok.insert(i, '[MASK]')
                print(f'INPUT_TO_BERT: {new_tok}')
                indexed_tokens = torch.tensor([tokenizer.convert_tokens_to_ids(new_tok)]).to('cuda')
                segment_ids = torch.tensor([[0 for x in range(indexed_tokens.shape[1])]]).to('cuda')
                #print(indexed_tokens)
                #print(segment_ids)
                with torch.no_grad():
                    predictions = model(indexed_tokens, segment_ids)
                predictions = predictions.cpu()
                res[i] = []
                for j in range(i, i+num_tokens):
                    #predictions.cpu()[0, j].numpy().argsort(axis = 1).T[-10:][::-1].T
                    predicted_indexes = predictions.cpu()[0, j].numpy().argsort()[-threshold:][::-1]
                    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes)
                    res[i].append(predicted_tokens)
                    print(predicted_tokens)
    return res

In [18]:
st = 'Магазинчик был маленьким, а мясник и его супруга очень большими, поэтому, когда к ним прибавились мы с женой, в лавке стало совсем тесно.'
tokenizer.tokenize(st)
#tokenizer.tokenize('содержат')

['М',
 '##ага',
 '##зин',
 '##чик',
 'был',
 'мал',
 '##ень',
 '##ким',
 ',',
 'а',
 'м',
 '##яс',
 '##ник',
 'и',
 'его',
 'су',
 '##пруга',
 'очень',
 'большим',
 '##и',
 ',',
 'поэтому',
 ',',
 'когда',
 'к',
 'ним',
 'при',
 '##ба',
 '##вили',
 '##сь',
 'мы',
 'с',
 'женой',
 ',',
 'в',
 'ла',
 '##в',
 '##ке',
 'стало',
 'сов',
 '##сем',
 'те',
 '##сно',
 '.']

In [19]:
#bottom_layer(st, [[' ' for x in range(1)]], tokenizer, 0, model, 10)

In [20]:
tokenizer.tokenize('был')

['был']

In [21]:
#tokenizer.convert_ids_to_tokens(tmp)

In [22]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [61]:
def middle_layer(tok_sent, verb, tokenizer, index, model, threshold, sent):
    # принимает на вход предложение, глагол, токенизатор, индекс, модель и threshold+sent; возвращает список форм глагола
    # на этом же слое должно происходить объединение результатов (есть ли гэппинг)
    # пока что всё делаем для одной формы глагола
    is_gapping = 0
    V_pos = []
    V_pos_res = ""
    cV_pos = "-1:-1"
    tok_verb = tokenizer.tokenize(verb)
    generated_words = bottom_layer(tok_sent, [tok_verb], tokenizer, index, model, threshold) # словарь
    #print(generated_words)
    for key in generated_words.keys():
        assert len(generated_words[key]) == len(tok_verb)
        assert len(generated_words[key]) > 0
        gapping_flag = 1
        for i in range(len(tok_verb)):
            if tok_verb[i] not in generated_words[key][i]:
                gapping_flag = 0
        if (gapping_flag == 1):
            is_gapping = 1
            V_pos.append(key)
            if (cV_pos == "-1:-1"):
                tmp = sent.rfind(verb)
                cV_pos = f"{tmp}:{tmp + len(verb)}"
    for elem in V_pos: #TODO: реализовать быстрый поиск позиции слова
        tmp = sent.rfind(tok_sent[elem])
        V_pos_res += f"{tmp}:{tmp} "
    return is_gapping, cV_pos, V_pos_res

In [62]:
def top_layer(sent, tokenizer, model, sentence_tokenizer, morph, threshold=10):
    # на вход - предложение, токенизатор, модель, threshold, токенизатор для предложений (BasicTokenizer)
    search_sent = sent.lower()
    tok_sent = tokenizer.tokenize(sent)
    tok_verb_sent = sentence_tokenizer.tokenize(search_sent)
    print(tok_verb_sent)
    verb_list = [word for word in tok_verb_sent if check_verb_in_tags(word, morph) == 'VERB']
    print(verb_list)
    for verb in verb_list:
        verb_pos = search_sent.rfind(verb)
        tok_prefix = tokenizer.tokenize(sent[:(verb_pos+len(verb))])
        index = len(tok_prefix) + 1
        is_gapping, cV_pos, V_pos = middle_layer(tok_sent, verb, tokenizer, index, model, threshold, sent)
        if (is_gapping == 1):
            return is_gapping, cV_pos, V_pos
    return 0, "-1:-1", "-1:-1"

In [63]:
def bottom_layer(tok_sent, verb_list, tokenizer, index, model, threshold):
    # получаем предложение, список глаголов, токенизатор, индекс конца слова;возвращаем "похожие" токенизированные слова
    # также подаём на вход модель; threshold - количество получаемых элементов
    #tok_sent = tokenizer.tokenize(sent) # пока что
    res = {}
    if (len(verb_list) == 1):
        verb = verb_list[0]  # список из k элементов (k - количество токенов)
        num_tokens = len(verb)
        print(f"{verb} --\n {index} --\n {tok_sent}")
        for i in range(index+2, len(tok_sent)):
            if (tok_sent[i].isalpha() or tok_sent[i].isdigit()):
                # TODO: добавить предыдущее слово
                    new_tok = tok_sent.copy()
                    # тогда перед этим всем вставляем num_tokens токенов
                    for _ in range(num_tokens):
                        new_tok.insert(i, '[MASK]')
                    indexed_tokens = torch.tensor([tokenizer.convert_tokens_to_ids(new_tok)]).to('cuda')
                    segment_ids = torch.tensor([[0 for x in range(indexed_tokens.shape[1])]]).to('cuda')
                    #print(indexed_tokens)
                    #print(segment_ids)
                    with torch.no_grad():
                        predictions = model(indexed_tokens, segment_ids)
                    predictions = predictions.cpu()
                    res[i] = []
                    for j in range(i, i+num_tokens):
                        #predictions.cpu()[0, j].numpy().argsort(axis = 1).T[-10:][::-1].T
                        predicted_indexes = predictions[0, j].numpy().argsort()[-threshold:][::-1]
                        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes)
                        res[i].append(predicted_tokens)
    return res

In [64]:
from pytorch_pretrained_bert import BasicTokenizer

In [65]:
from extra import check_verb_in_tags

In [66]:
tmp = BasicTokenizer(do_lower_case=False)
st = 'Мастер выполняет необрезной маникюр, для придания формы используются особые щадящие мелкозернистые пилочки, а для дезинфекции эффективные антисептики.'
wtmp = tmp.tokenize(st)
tst = tokenizer.tokenize(st)

In [67]:
top_layer(st, tokenizer, model, tmp, morph)

['мастер', 'выполняет', 'необрезной', 'маникюр', ',', 'для', 'придания', 'формы', 'используются', 'особые', 'щадящие', 'мелкозернистые', 'пилочки', ',', 'а', 'для', 'дезинфекции', 'эффективные', 'антисептики', '.']
['выполняет', 'используются']
['вы', '##по', '##л', '##няет'] --
 8 --
 ['М', '##аст', '##ер', 'вы', '##по', '##л', '##няет', 'не', '##об', '##рез', '##ной', 'ма', '##ник', '##ю', '##р', ',', 'для', 'при', '##дания', 'формы', 'используются', 'ос', '##обы', '##е', 'щ', '##ад', '##ящие', 'м', '##ел', '##ко', '##зер', '##нис', '##тые', 'п', '##ило', '##чки', ',', 'а', 'для', 'де', '##зин', '##фе', '##кции', 'э', '##ффект', '##ивные', 'ан', '##ти', '##се', '##п', '##тики', '.']
['используются'] --
 22 --
 ['М', '##аст', '##ер', 'вы', '##по', '##л', '##няет', 'не', '##об', '##рез', '##ной', 'ма', '##ник', '##ю', '##р', ',', 'для', 'при', '##дания', 'формы', 'используются', 'ос', '##обы', '##е', 'щ', '##ад', '##ящие', 'м', '##ел', '##ко', '##зер', '##нис', '##тые', 'п', '##ило', '

(1, '56:68', '110:110 126:126 ')

In [360]:
tmp.tokenize

<bound method BasicTokenizer.tokenize of <pytorch_pretrained_bert.tokenization.BasicTokenizer object at 0x7f6707dc3550>>

In [310]:
set([x.tag.POS for x in morph.parse('овгз'.lower()) if x.score > 0.1])

{'NOUN'}

In [301]:
tmp.tokenize(st)

['например',
 ',',
 'с',
 'помощью',
 'штатива',
 'можно',
 'выполнять',
 'съемку',
 'с',
 'автоматическим',
 'началом',
 'записи',
 'или',
 'снимать',
 'объекты',
 'при',
 'недостаточном',
 'освещении',
 ',',
 'например',
 'феиерверки',
 'и',
 'ночные',
 'виды',
 '.']

In [182]:
middle_layer(tst, 'подтвердили', tokenizer, 17, model, 20, st)

##ые
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с', '##менов', 'под', '##тве', '##рди', '##ли', 'з', '##елен', '##ые', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'по', '##яса', ',', 'шест', '##еро', '-', 'же', '##лт', '##ые', '.']
,
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с', '##менов', 'под', '##тве', '##рди', '##ли', 'з', '##елен', '##ые', 'по', '##яса', ',', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'шест', '##еро', '-', 'же', '##лт', '##ые', '.']
-
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с', '##менов', 'под', '##тве', '##рди', '##ли', 'з', '##елен', '##ые', 'по', '##яса', ',', 'шест', '##еро', '-', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'же', '##лт', '##ые', '.']
.
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с

(0, '-1:-1', '')

In [131]:
ltmp = [word for word in wtmp if check_verb_in_tags(word, morph) == 'VERB']

In [132]:
st = st.replace('й', 'и').replace('ё', 'е')
for elem in ltmp:
    print(st.rfind(elem))

13
50
92


In [133]:
tst = tokenizer.tokenize(st)

In [134]:
ltmp

['видел', 'был', 'боли']

In [135]:
for elem in ltmp:
    print(st[:(st.rfind(elem) + len(elem))])

Он как будто видел
Он как будто видел что-то в глубине, и взгляд его был
Он как будто видел что-то в глубине, и взгляд его был рассеянным, а выражение лица — полным боли


In [137]:
middle_layer(tst, 'был', tokenizer, 4, model, 10, st)

##л
=====
-
=====
то
=====
в
=====
,
=====
и
=====
##гляд
=====
его
=====
был
=====
,
=====
а
=====
##ражение
=====
[UNK]
=====
##ным
=====
.
=====


(1, (50, 53), '85:85 ')